In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import os

from astropy.io import fits
from astropy.table import Table, join
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

# Download Core Galaxy Sample

This notebook:

- records the data download process for UKIDSS and SDSS
- plots the overall galaxy sample features

## Download Approach

Neither SDSS nor UKIDSS has a complete copy of the other on their servers. Therefore, I need to use remote access to BOTH services and then perform a join.


I tried downloading UKIDSS and SDSS via the CDS X-Match service available from cdsxmatch.u-strasbg.fr/xmatch. The latest available catalogs on X-Match via Vizier were UKIDSS-DR9 LAS+GCS+DXS and SDSS DR12. However, the download fell over above a handful of rows.


**Current approach:**
1. Load the NSA into Topcat from FITS (with useful columns only?)
2. Get UKIDSS cross-matched (5 arcsecs) with NSA via Topcat remote Vizier crossmatch (X button)
3. Download OSSY subtables via the Wordpress release website and join into one master table
4. Locally match (matchsticks button) NSA/UKIDSS and OSSY_joined (5 arcsecs)

The UKIDSS table used was `UKIDSS-DR9 LAS` on Vizier, available from [here](http://vizier.u-strasbg.fr/viz-bin/VizieR-3?-source=II/319/las9) with a schema.

### 1. Load NSA into Topcat

### 2. Get UKIDSS cross-matched with NSA

### 3. Download OSSY Subtables

In [ ]:
ossy_dir = 'catalogs/ossy/raw'
### TODO use command line to download OSSY catalogs

In [ ]:
emission_flux = Table.read(os.path.join(ossy_dir, 'OSSY_emission_flux_errors.fits'))
emission_ew = Table.read(os.path.join(ossy_dir, 'OSSY_emission_aon_ew_sky.fits'))
continuum_qa = Table.read(os.path.join(ossy_dir, 'OSSY_quality_assessment_continuum.fits'))
emission_qa = Table.read(os.path.join(ossy_dir, 'OSSY_quality_assessment_emission.fits'))
ossy_sdss = Table.read(os.path.join(ossy_dir, 'OSSY_SDSS_parameters.fits'))
assert len(ossy) == len(emission_flux) == len(emission_ew) == len(continuum_qa) == len(emission_qa) == len(ossy_sdss)

In [ ]:
joined_dir = 'catalogs/ossy/joined'
if not os.path.isdir(joined_dir):
    os.mkdir(joined_dir)
# ossy.to_csv(os.path.join(joined_dir, 'ossy_joined.csv'), index=False)  # takes a minute, large csv
ossy.write(os.path.join(joined_dir, 'ossy_joined.fits'), overwrite=True)

### 4. Use TOPCAT to join OSSY to the rest.

Save to `catalogs/nsa_v1_0_1_ossy_ukidss_dr9_las.fits`

### 5. Remote cross-match into WISE

 <img src="figures/allwise_crossmatch.png" alt="allwise crossmatch" height="400" width="300"> 

Save to `catalogs/nsa_v1_0_1_ossy_ukidss_dr9_las_allwise.fits`

In [6]:
wise_cols = ['AllWISE', 'RAJ2000', 'DEJ2000', 'eeMaj', 'eeMin', 'eePA', 'W1mag', 'W2mag', 'W3mag', 'W4mag', 'Jmag', 'Hmag', 'Kmag', 'e_W1mag', 'e_W2mag', 'e_W3mag', 'e_W4mag', 'e_Jmag', 'e_Hmag', 'e_Kmag', 'ID', 'ccf', 'ex', 'var', 'qph', 'pmRA', 'e_pmRA', 'pmDE', 'e_pmDE', 'd2M', 'angDist_wisex']

### Add Ellison L_IR where possible

Add the inferred total IR luminosity from [this](https://academic.oup.com/mnras/article/455/1/370/984265#78675740) paper where possible

In [4]:
ellison_data_loc = "https://oup.silverchair-cdn.com/oup/backfile/Content_public/Journal/mnras/455/1/10.1093_mnras_stv2275/2/stv2275_Supplementary_Data.zip?Expires=2147483647&Signature=fUPDeu3Bxkd8mbg7RtlctfpE~rK8xoS4OtBPDUED4jhy8caDLkmhuAbYxcunNyZGJXLHzCctsusHedfnrVpmcW3yKosbUBIzMQQAZCWXKWa22A8bdr5meMv1M~zyC8GF0a7gERKRRGb1fsy3PQL9fPbsxs2dT3OlAoQdkmkLT0lYCUbUc~IIbgcPaypbqzUU1HcYvryps1zbdrH0rZcU8oeyjp0Km0t-yM-esyWRm8l7l8Ncr~hDutv4MjSensREAuuJiK2qdgPz27we-9S~HPbFLte2Sqt4QYdZXHubOyNMEIN-CjudhuvfNGyHzl2kmTvS1wPubKXLnspdj2S7MQ__&Key-Pair-Id=APKAIE5G5CRDK6RD3PGA"

In [15]:
download_loc = 'catalogs/ellison_inferred_fir/ellison_ir.zip'

In [16]:
!wget "$ellison_data_loc" -O  "$download_loc" 

--2018-07-11 10:24:11--  https://oup.silverchair-cdn.com/oup/backfile/Content_public/Journal/mnras/455/1/10.1093_mnras_stv2275/2/stv2275_Supplementary_Data.zip?Expires=2147483647&Signature=fUPDeu3Bxkd8mbg7RtlctfpE~rK8xoS4OtBPDUED4jhy8caDLkmhuAbYxcunNyZGJXLHzCctsusHedfnrVpmcW3yKosbUBIzMQQAZCWXKWa22A8bdr5meMv1M~zyC8GF0a7gERKRRGb1fsy3PQL9fPbsxs2dT3OlAoQdkmkLT0lYCUbUc~IIbgcPaypbqzUU1HcYvryps1zbdrH0rZcU8oeyjp0Km0t-yM-esyWRm8l7l8Ncr~hDutv4MjSensREAuuJiK2qdgPz27we-9S~HPbFLte2Sqt4QYdZXHubOyNMEIN-CjudhuvfNGyHzl2kmTvS1wPubKXLnspdj2S7MQ__&Key-Pair-Id=APKAIE5G5CRDK6RD3PGA
Resolving oup.silverchair-cdn.com (oup.silverchair-cdn.com)... 52.85.58.217, 52.85.58.212, 52.85.58.116, ...
Connecting to oup.silverchair-cdn.com (oup.silverchair-cdn.com)|52.85.58.217|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6353471 (6.1M) [application/zip]
Saving to: ‘catalogs/ellison_inferred_fir/ellison_ir.zip’

catalogs/ellison_in 100%[===================>]   6.06M  --.-KB/s    in 0.08s   



In [17]:
# TODO unzip the file

In [ ]:
# TODO cross-match here, rather than at the end of basic_sample_details